In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 34.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.1 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=c591c49e27c168b495850bfb6f2a84c278fa54f1fad31993bef7c82eef389710
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [20]:
import os
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from deepface import DeepFace
from pathlib import Path

In [28]:
save_path="/content/faces"
database="/content/database"
os.makedirs(save_path,exist_ok=True)
os.makedirs(database,exist_ok=True)

In [9]:
model=YOLO("yolov8n-face.pt")

In [11]:
def crop_image(input_img,box_coord,save_path):
    img=Image.open(input_img)
    cropped_img=img.crop(box_coord)
    if input_img.endswith(".jpg"):
      cropped_img.save(save_path+"/image.jpg")
    elif input_img.endswith(".png"):
      cropped_img.save(save_path+"/image.png")
    else:
      cropped_img.save(save_path+"/image.jpeg")
    cropped_img.show()

In [26]:
def face_recognition(input_image):

    result=model.predict(source=input_image,save=True,conf=0.5)

    bbox=result[0].boxes.xyxy[:].tolist()

    for boxes in bbox:
      box_coord=boxes
      crop_image(input_image,box_coord,save_path)

    res=[]
    for img in os.listdir(save_path):
      face_path=os.path.join(save_path,img)

      if face_path.endswith(".jpg"):
        try:
            # Face recognition with DeepFace
            result = DeepFace.find(img_path=face_path, db_path=database, model_name="VGG-Face", enforce_detection=False)

            if result and len(result[0]["identity"]) > 0:
                file_path = Path(result[0]["identity"][0])
                recognized_name = file_path.parent.name
                res.append(recognized_name)
            else:
                recognized_name = "Unknown"
        except Exception as e:
            recognized_name = ""

    return res


In [27]:
input_image="/content/IMG-20250128-WA0025.jpg"
names=face_recognition(input_image)
print(names)


image 1/1 /content/IMG-20250128-WA0025.jpg: 640x384 1 face, 122.2ms
Speed: 3.0ms preprocess, 122.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)
Results saved to runs/detect/predict2
25-02-01 13:29:15 - Searching /content/save/image.jpg in 3 length datastore
25-02-01 13:29:16 - find function duration 1.1148550510406494 seconds
['Abhay']
